In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing

In [2]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"

films = pd.read_csv(film_csv, index_col=None, header=0)
display(films.head())

trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_meta.drop(trailers_meta.columns[trailers_meta.columns.str.contains('_y',case = False)],axis = 1, inplace = True)
trailers_meta.drop(trailers_meta.columns[trailers_meta.columns.str.contains('_x',case = False)],axis = 1, inplace = True)
display(trailers_meta.head())
trailers_meta.to_csv(trailers_csv)

,id,title,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties
0,60432,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д..."
1,64219,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес..."
2,65407,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл..."
3,75149,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""..."
4,75629,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...","{ ""Аудитория"" : [""для детей 13-16 лет"",""9 лет""..."


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,0,0,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,1,1,1,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,2,2,2,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,3,3,3,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,4,4,4,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [3]:
# WARNING: не запускать, т.к. скрипт уже был использован и испортит данные

# trailers_meta['properties'].to_list()
# cols = ["Место действия", "Качества", "Время действия", "На основе", "Аудитория", "Настроение", "Поджанр", "Про что", "Тематика"]

# places = []
# qualities = []
# times = []
# mains = []
# audiences = []
# moods = []
# subgenres = []
# abouts = []
# themes = []

# for i in range(0, len(trailers_meta)):
#     trailers_info = json.loads(trailers_meta.iloc[i]['properties'])
    
#     try:
#         place = trailers_info['Место действия']
#     except:
#         place = ['не определено']
#     places.append(place)
    
#     try:
#         quality = trailers_info['Качества']
#     except:
#         quality = ['не определено']
#     qualities.append(quality)
    
#     try:
#         time = trailers_info['Время действия']
#     except:
#         time = ['не определено']
#     times.append(time)
    
#     try:
#         main = trailers_info['На основе']
#     except:
#         main = ['не определено']
#     mains.append(main)
    
#     try:
#         audience = trailers_info['Аудитория']
#     except:
#         audience = ['не определено']
#     audiences.append(audience)
    
#     try:
#         mood = trailers_info['Настроение']
#     except:
#         mood = ['не определено']
#     moods.append(mood)
    
#     try:
#         subgenre = trailers_info['Поджанр']
#     except:
#         subgenre = ['не определено']
#     subgenres.append(subgenre)

#     try:
#         about = trailers_info['Про что']
#     except:
#         about = ['не определено']
#     abouts.append(about)
    
#     try:
#         theme = trailers_info['Тематика']
#     except:
#         theme = ['не определено']
#     themes.append(theme)
    

# df = pd.DataFrame({
#     'content_id': trailers_meta['content_id'],
#     'Place of act': places,
#     'Qualities': qualities,
#     'Time of act': times,
#     'Based on': mains,
#     'Audience': audiences,
#     'Mood': moods,
#     'Subgenre': subgenres,
#     'About': abouts,
#     'Theme': themes
# })

# display(df.head())
# display(trailers_meta.head())

# trailers_df = pd.merge(trailers_meta, df, left_on='content_id', right_on='content_id', how='left')
# trailers_df = trailers_df.drop_duplicates(subset=['content_id'])
# trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
# display(trailers_df.head())

# trailers_df.to_csv(trailers_csv)

In [4]:
info = {}
min_len = 100
max_len = 0
del_array_name = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_meta)):
    name = trailers_meta.iloc[i]['trailers_name']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info_df = info_df.iloc[: , 3:]
        info[name] = info_df
    except:
        del_array_name.append(name)

In [5]:
print(del_array_name, len(del_array_name))

['0abceea32472c573ba7af504fd00fd', '2fa0ae09168251b817ab94cd60ce8a', 'fcbee265edc0503ad53a28c983c5e0', 'fde0e8cf8408f87356ac8fb722f481', 'f71302bf45d805cc80d4f8d5c41adc', 'a1c32f6263e9b6b26cdcbe208fa209', '194e610175ced32afa97ed202589a9', '1f01cfdb79b08ffde79eecd37760c1', '8bd3f4ebba62fa13ace809ba0efb68', '8d4386805fff1944baafba236650dd', '268fa5408e3080413d87557c7a5a40', '31eaaadbe52b0d65e819af4e4ae908', '1b988cc0438af387aeb16e8d1d36aa', '27d95cd4accb1f17508fcebce4bb3e', 'c5176f62d9e78b7508e91feed59e73', '32818249a83e2412ce506a131c43e0', '29882a40ff24e672596d99ff64ae17', '5c383cdc4f9194234194d3b3d807d3', '1cb6a48e086ab54867ea5ae89b1713', '967c11c3edbe5eaaed5b9bb2cd110c', '634679bb639198d034489b96eeea50', 'ab3bf566de4a72493b5d68ef2dc962', '921ce8b1f6a9ccc8aa0384d08c1522', '908293f3c6eb04489efc9ddc70b224', '0c9d3593f104130849949f5f384a4e', '6b1e914b7eaa6b0cc99285327db84e', '8e1f23602bfa26752e8a81c59d37b6', 'c1cec0a87f7c0128771c4f472724da', 'ac82f9a9ba4735c94f72471165f84b', '0e696eb09481

In [6]:
print(len(info))

912


In [16]:
def make_session(df, start_ind = 0, session_len = 10):
    names = [df.iloc[start_ind]['trailers_name']]
    
    curr_info = info[names[0]]
    bright = np.mean(curr_info['brightness'].to_list())
    color = np.mean(curr_info['colorfulness'].to_list())
    tempo = np.mean(curr_info['tempo'].to_list())
        
    time_of_act = json.loads(df.iloc[start_ind]['Time of act'].replace("\'", "\""))
    emotions = json.loads(df.iloc[start_ind]['emotions'].replace("\'", "\""))
    emotions_neu = emotions['neutral']
    emotions_neg = emotions['negative']
    emotions_pos = emotions['positive']
    emotions_skip = emotions['skip']
    emotions_speech = emotions['speech']
    genres = json.loads(df.iloc[start_ind]['Subgenre'].replace("\'", "\""))
    mood = json.loads(df.iloc[start_ind]['Mood'].replace("\'", "\""))
    about = json.loads(df.iloc[start_ind]['About'].replace("\'", "\""))
    
    i = 0
    while i < len(df):
        if len(names) == session_len:
            break
            
        name = df.iloc[i]['trailers_name']
        if name not in names:
            i_emo = json.loads(df.iloc[i]['emotions'].replace("\'", "\""))
            i_genre = json.loads(df.iloc[i]['Subgenre'].replace("\'", "\""))
            i_mood = json.loads(df.iloc[i]['Mood'].replace("\'", "\""))
            i_about = json.loads(df.iloc[i]['About'].replace("\'", "\""))
            i_time = json.loads(df.iloc[i]['Time of act'].replace("\'", "\""))
            if name not in del_array_name:
                i_curr_info = info[name]
                i_bright = np.mean(curr_info['brightness'].to_list())
                i_color = np.mean(curr_info['colorfulness'].to_list())
                i_tempo = np.mean(curr_info['tempo'].to_list())
                if (
                    np.linalg.norm(emotions_neu - i_emo['neutral']) < 0.04 and
                    np.linalg.norm(emotions_neg - i_emo['negative']) < 0.04 and
                    np.linalg.norm(emotions_pos - i_emo['positive']) < 0.04 and
                    len(list(set(i_genre) & set(genres))) > 0 and
                    len(list(set(i_mood) & set(mood))) > 0 and
                    len(list(set(i_about) & set(about))) > 0 and
                    len(list(set(i_time) & set(time_of_act))) > 0 and
                    np.linalg.norm(bright - i_bright) < 0.04 and
                    np.linalg.norm(color - i_color) < 0.04 and
                    np.linalg.norm(tempo - i_tempo) < 0.04
                ):
                    names.append(name)
                    genres = list(set(i_genre) | set(genres))
                    mood = list(set(i_mood) | set(mood))
                    about = list(set(i_about) | set(about))
                    time_of_act = list(set(i_time) | set(time_of_act))
                    emotions_neu = np.mean([emotions_neu, i_emo['neutral']])
                    emotions_neg = np.mean([emotions_neg, i_emo['negative']])
                    emotions_pos = np.mean([emotions_pos, i_emo['positive']])
                    emotions_skip = np.mean([emotions_skip, i_emo['skip']])
                    emotions_speech = np.mean([emotions_speech, i_emo['speech']])
                    bright = (bright + i_bright) / 2
                    color = (color + i_color) / 2
                    tempo = (tempo + i_tempo) / 2
                    i = 0
                else:
                    i += 1
            else:
                i += 1
        else:
            i += 1
        
    return names
    

sessions = []
j = 0
while j < len(trailers_meta):
    print(j)
    session = make_session(trailers_meta, j)
    if len(session) == 10:
        print("append", len(sessions), session)
        sessions.append(session)
    j += 1
    if len(sessions) > 500:
        break;
    
sess_df = pd.DataFrame(sessions)
display(sess_df.head())
sess_df.to_csv("/Volumes/Seagate/natasha-diploma/sessions1.csv")

0
append 0 ['403a42b604f504069c41d39c5a4fc2', '0dbe493275a25285829b979316d566', '9aecf32a60e4e0f7f967caecb6c798', '23ee5640ec588ffaa78afa3268b04a', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd44ad5cc2d63441ebf437eccb5', '26b3f811a6b9ed75d3cc5198105fa4', '06132809afbe1c781e1b9605452081', '67354be5f6f02a74e7661f9529daeb']
1
2
3
4
append 1 ['d66440e7524c0a8b9b206fbb7295ba', 'a6d83ff19ed58d43644ca319a2c5d4', 'a98af02d324af4f43b7a42c82b73d0', '62e8a540d2321f61f2b47dd26103e4', '807d308f6b0d8df70b5ffbd5b88f38', '7f778c07837b7646904a091a481145', '76dc726c7fb950a0fff79073b229f6', '930e6d99ffcb060d6b030d4cc90680', '4b7669ea29a6770d7ee65e57b34e63', '44befb98302cfa436627fa3af38642']
5
append 2 ['231b80c884b2f1390e17f09a187889', 'bd6707bb04cfb8880a5f783c85be20', 'd6159553e41857fe23df9d3e746284', 'c0314a446fbef65ce9626b037a70da', '961c336b0c2fa04c883c639dfee1ba', '514f3ee1f00df05dc3900251972245', '2c72ca8794dd4eb8f5ae90ca0dce24', 'c292588c17f2fb55db17a98774ef07', '40fa5

59
60
61
62
63
64
65
append 25 ['ba4e9f5493b5f4638a751c734773fb', '644bf6d0bdf6c9cddf71f5ab4cf7d4', '7cb4804352fe8c46fe686d592587b6', '4774caaf01df3d5d982e0cb77b1a13', '92259264d00abaec349763f17293fd', '75f3bae4a338b5832508e7bab6f378', '37ac3c01b526131ef198faca2cd26c', 'd9fade6ab23b9f083681c300839a07', '0c14d1263d70aae94c0464ce236ccd', 'd1d3ab674025e8458cc103d3c7a887']
66
append 26 ['c739a34abff60f3677b0daf54cf077', 'c69ed1ba252f76f2f779674f524b3e', '40f0e24908cd965e83d2a52838f816', 'c34235582b76806e1ccc20d126eac7', '63da1e82f584e99c7b3dd6d7e0ded6', '7d56b2f4cf9c8042ee7b2c4df50f66', 'dcf30d1e701e9828b33d67c9dac6f9', '8a2483e11828332a9e2fc8659ae38f', '86ae94aaa9a2e1eae23183d8040d41', '0f462e6bbd719e012902d8814d0e35']
67
append 27 ['b5054678d47bb23ce64353afe84df7', '80b59a49c3ab2430b86d3c0919e5a3', 'd081888b9f0f96063f5f46a068b1a0', '8a4d7d84e0e821e55719ad5eead70b', '24343e24b68499240dd543a8786271', '972821d03656142f33707487f51b66', 'd6d45793f5b093e8bdf3d8ae4952b6', '28731720ee3330b7107ce

append 49 ['2c5bf20534b80ee47bed9a90b405a0', '2adfc14bbc0fba5b5d616d6fcfe3de', '274869b912b256df69b9e7c5c0cb11', 'f456635bb3d2567a3a61133fa9dd89', '294b1604461720bdf36de6d46d96b3', '227e9faf3bd31e0f87eb4afd72a69d', '6d0bb6959972ac92c10d7445982a94', '3b67fe7d2253d0db8bbd65b56fbcd4', '1797d412a657f966ab24e78402f845', '87712a41c2abd488fcd45756601f92']
115
116
117
118
119
120
121
122
append 50 ['c775ad98eb2cb98e5dd370867bd0cf', '9aecf32a60e4e0f7f967caecb6c798', '67354be5f6f02a74e7661f9529daeb', '403a42b604f504069c41d39c5a4fc2', '23ee5640ec588ffaa78afa3268b04a', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd44ad5cc2d63441ebf437eccb5', '26b3f811a6b9ed75d3cc5198105fa4', '06132809afbe1c781e1b9605452081']
123
124
125
append 51 ['26ba677f7ff11ac529f5f69c54cb0b', 'f65c8906c07b627a760a0671de7144', '123562dc6526434b4094ce55501991', 'a91796240352e8f4c383dc5d630e61', '703ab83bbbf120aff754b766b6daed', 'e4bb58ebea5274f980f7e9bcdaafaa', '9146008bb07ccbfba44794ee20e00a', '8982

176
append 72 ['9aecf32a60e4e0f7f967caecb6c798', '403a42b604f504069c41d39c5a4fc2', '67354be5f6f02a74e7661f9529daeb', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd44ad5cc2d63441ebf437eccb5', '26b3f811a6b9ed75d3cc5198105fa4', '23ee5640ec588ffaa78afa3268b04a', '06132809afbe1c781e1b9605452081', 'e3e5642e76f638dd918e718dfefe3a']
177
178
179
append 73 ['312d9811543f24aeba98608b5ae7a8', '3095fc6d726de6c590a3b7ffc0cf01', '9914764b34cff2aac25a82a33d5494', 'd95b26f119df48eadb34cd4746c69a', '09887453861c8d4e951ceca579ebfb', 'ed34762a8e3183ccff945f0d964107', '6fcff414343456465369b05f6851eb', '05c92546aa14ea6148088b7c713be4', 'bac38c80bef68386106932528fcd50', '159a6475dde4d3afbb6447326500f8']
180
181
182
append 74 ['d42c28fdec29493cc826c520e67b51', '1c3a0eaf4e34f3a847dc53205e31bc', '08712d35cf70496630e16b24dae005', '11670c9e7db3805487f2d4f841459e', 'd9fade6ab23b9f083681c300839a07', '7cb4804352fe8c46fe686d592587b6', '92259264d00abaec349763f17293fd', '02817e267c3e407bc7b2

224
225
226
227
228
append 95 ['c7f809aab571e9acdf799a78c5ef61', '6037867d48fd6141c6449f160788b0', '2c78140ecf16136e58af934792b4cf', '19891536f4e998fc5f0cbbd0792350', '3d558c3e37ccb70937b171aeb67bbb', '67f32aff5488778cb4bbb6b21acc7d', '51875c38944a44301109690e44d182', '9bc9eb659d4a054ff12b6adbb89dd3', '01bc55dad49dc1dfa856c85edbf6d1', '84b95e6532916ade37a5b25a3dbfce']
229
append 96 ['367bc88c1eb231dbc494e6822a0552', '78ebff6bc5b4fcb86cff95fc675050', '001db9af2dcc57d45d653aeb33171e', 'fe3ed77a61a83c34e9cc8abc7f5ffa', '5bdc48f75195945de3bf34b5b17d15', 'f2d541670a085da7cb5713d79b7666', '294b1604461720bdf36de6d46d96b3', '2adfc14bbc0fba5b5d616d6fcfe3de', '2c5bf20534b80ee47bed9a90b405a0', '274869b912b256df69b9e7c5c0cb11']
230
append 97 ['061a59839837a123a916a70947f075', '772c5a2f027d06d8b7046a08374cdc', 'ad9ad1a92ab6ca25c67507bf3d7c75', 'e82d0fb871a83d8983ef7a2fc99a37', '3a5b542f914f6bbd8c7a9e28b2153d', '10784fd4ab945005a939388b1e329f', '8982bc4a120cde857075742b51fbda', 'bc6d8d64d120ac48e6a5

...

677
678
append 285 ['3f7278f8034888df8dffcc2c753d8a', 'dcf30d1e701e9828b33d67c9dac6f9', '86ae94aaa9a2e1eae23183d8040d41', '7d56b2f4cf9c8042ee7b2c4df50f66', '8a2483e11828332a9e2fc8659ae38f', '8d3877a1b26b10642d125ec4fc4c9b', '105dac1b6a9ca37eb8067bbb80260a', 'e5bca888fec7f8c0116db0002a7077', 'ae90166aa81ab2bf878dace61a5620', '433b9cff26f4d11d4af6889dc0fba2']
679
680
append 286 ['d6f4ce2516113cb955c447385ac3b5', 'efc61b10365f5fcdd4c7c5e6a8684a', '9e05015e2042286d7c9d21f214425e', '49e57c1ad425edf22eed61b598c0ec', 'fcdebb544d991f71e3abe0c5b09af2', '58a923b314735ae6f938d4c51c85f6', '637463972c3f9869044cdffaf98c8d', '8f762e69cf2e7a36ee0e75a0d0eae5', '29cc4ae075320669bcf33f11933740', '06d4e0812e40c50f93b3407878e744']
681
682
append 287 ['b99c739ddbbe2dcbc071d89dc2a18b', 'c739a34abff60f3677b0daf54cf077', 'c69ed1ba252f76f2f779674f524b3e', 'c34235582b76806e1ccc20d126eac7', '40f0e24908cd965e83d2a52838f816', '63da1e82f584e99c7b3dd6d7e0ded6', '86ae94aaa9a2e1eae23183d8040d41', 'dcf30d1e701e9828b33d6

append 308 ['d1a994082b70e143ebe89de97700a8', '5e2f9f78972916d007e7c826814c77', '3b67fe7d2253d0db8bbd65b56fbcd4', 'f5358108d18cec343cbdac0a80880e', '1797d412a657f966ab24e78402f845', '22c35dd51c4c6b1bd62abcdeebe769', '227e9faf3bd31e0f87eb4afd72a69d', '6d0bb6959972ac92c10d7445982a94', '87712a41c2abd488fcd45756601f92', '274869b912b256df69b9e7c5c0cb11']
724
append 309 ['830a1ec2650f409bdd2fb9593d332f', '403a42b604f504069c41d39c5a4fc2', '0dbe493275a25285829b979316d566', '8640dd4d602b0809ef6395268a963f', '06132809afbe1c781e1b9605452081', '5b150a1cae009459493bd0fe5c7b3e', 'e291f8ae1de4a38c71027e9c91d51c', '34cfd44ad5cc2d63441ebf437eccb5', '26b3f811a6b9ed75d3cc5198105fa4', '23ee5640ec588ffaa78afa3268b04a']
725
726
727
append 310 ['368753dd516830d11a2afea8b8352a', 'e762f208419b7449c0731037262bb3', '7d20e50d5f9a014a6078c92c5f17ad', '3eb635e807cc6a8484531b00486f71', '8c7ff3370563e8ad6e2fb534a1ede7', 'b1b047cdaf1f69c31686b612f9e39e', 'bc800bb08fd91d219bf944fddd37aa', 'bd6707bb04cfb8880a5f783c85be2

append 332 ['be721756a2cd9a29d688acd3c2d613', '7bd3e91e2fb33040c729642921687f', '2d24fe1baec3caec802a87dd8e4fb8', '9146008bb07ccbfba44794ee20e00a', 'b53ef4ced60081ea54e126c90a4480', '85dee27066a189d8ade5c6f71069a7', '703ab83bbbf120aff754b766b6daed', '123562dc6526434b4094ce55501991', '8982bc4a120cde857075742b51fbda', '10784fd4ab945005a939388b1e329f']
799
append 333 ['ed34762a8e3183ccff945f0d964107', '4dcb7ccb49d0a9f928c94d459e2ccd', '6fcff414343456465369b05f6851eb', '4247088ee2239f9885a836fe4735cb', '41b772a5b18bc724562bfb60f4f3ac', '34cfd44ad5cc2d63441ebf437eccb5', 'e291f8ae1de4a38c71027e9c91d51c', '5b150a1cae009459493bd0fe5c7b3e', '23ee5640ec588ffaa78afa3268b04a', '26b3f811a6b9ed75d3cc5198105fa4']
800
801
802
803
append 334 ['13a5561390bcb4f3fd0c0b5ea33083', '153068b35776cd04068215f6d45aa4', 'e2f15819192cded77ed1825a129c0c', 'fa9b3a8b46d9f601f099b32b481978', '23ee5640ec588ffaa78afa3268b04a', '3e3bc4db68ecafbab3d01b38bce6c5', 'ea4f9a22de6d23ff14b645e975230e', '87de245b87c3c2ec94b13cef2

862
863
864
865
866
867
868
869
870
append 358 ['30cee6c7e23b1ed4068b1b1456d8ad', 'd50faca942c93b02863dc4cc86a839', '2530d7a6ecf471f695a2ef7a8a4191', '914880359e59dbe7b7e60c4ddc82f4', 'd79747c567207447a95ed00f2533bd', '4247088ee2239f9885a836fe4735cb', 'ff72d38f6672324e852d2cd1e089b7', 'e2791858e10f8f17bbb8ceb8d7e86b', '159a6475dde4d3afbb6447326500f8', '7d20e50d5f9a014a6078c92c5f17ad']
871
872
append 359 ['0dd117d13f68304b08ab6c5b6af641', '1301d8c7c77005600f5a62944737a7', '05b7120042d9597d422c48ba6341b8', 'bc210a47f0102c49f93bcff2232b2d', '92259264d00abaec349763f17293fd', 'ba4e9f5493b5f4638a751c734773fb', '7cb4804352fe8c46fe686d592587b6', '75f3bae4a338b5832508e7bab6f378', '4774caaf01df3d5d982e0cb77b1a13', '644bf6d0bdf6c9cddf71f5ab4cf7d4']
873
874
875
876
append 360 ['24343e24b68499240dd543a8786271', '972821d03656142f33707487f51b66', '80b59a49c3ab2430b86d3c0919e5a3', '28731720ee3330b7107cec47168b8c', 'd6d45793f5b093e8bdf3d8ae4952b6', 'dad45a9302ad916f6901b86a744cdd', 'a252ade2bfa4c2d4272

KeyError: '0abceea32472c573ba7af504fd00fd'

In [ ]:
trailers_meta.to_csv(trailers_csv)

In [17]:
sess_df = pd.DataFrame(sessions)
display(sess_df.head())
sess_df.to_csv("/Volumes/Seagate/natasha-diploma/sessions1.csv")

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,0dbe493275a25285829b979316d566,9aecf32a60e4e0f7f967caecb6c798,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,26b3f811a6b9ed75d3cc5198105fa4,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb
1,d66440e7524c0a8b9b206fbb7295ba,a6d83ff19ed58d43644ca319a2c5d4,a98af02d324af4f43b7a42c82b73d0,62e8a540d2321f61f2b47dd26103e4,807d308f6b0d8df70b5ffbd5b88f38,7f778c07837b7646904a091a481145,76dc726c7fb950a0fff79073b229f6,930e6d99ffcb060d6b030d4cc90680,4b7669ea29a6770d7ee65e57b34e63,44befb98302cfa436627fa3af38642
2,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,d6159553e41857fe23df9d3e746284,c0314a446fbef65ce9626b037a70da,961c336b0c2fa04c883c639dfee1ba,514f3ee1f00df05dc3900251972245,2c72ca8794dd4eb8f5ae90ca0dce24,c292588c17f2fb55db17a98774ef07,40fa578cd222749786f5aa816ad3c5,7e2582b83c27340f388fdce46a9209
3,d50faca942c93b02863dc4cc86a839,2530d7a6ecf471f695a2ef7a8a4191,f3153c49fddebe52085185876b61c5,914880359e59dbe7b7e60c4ddc82f4,4247088ee2239f9885a836fe4735cb,9914764b34cff2aac25a82a33d5494,6fcff414343456465369b05f6851eb,41b772a5b18bc724562bfb60f4f3ac,34cfd44ad5cc2d63441ebf437eccb5,e291f8ae1de4a38c71027e9c91d51c
4,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,23ee5640ec588ffaa78afa3268b04a,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb,9aecf32a60e4e0f7f967caecb6c798,403a42b604f504069c41d39c5a4fc2,209f1d1349a473f3eb0b285140c70a,c775ad98eb2cb98e5dd370867bd0cf
